# Linear Programming Tutorial: 

## 1) The Furniture Factory Problem

## Objective and Prerequisites

The goal of this Jupyter Notebook is to describe the Furniture Factory Problem discussed in the
video: *Chapter 6, Modeling and Solving Linear Programming Problems*, of the [LP Tutorial](https://www.gurobi.com/resource/mathematical-programming-tutorial-linear-programming/).
The Furniture Factory Problem is formulated as a linear programming problem using the Gurobi Python API. The Furniture Factory Problem has been taken from the Saul Gass book *An illustrated Guide to Linear Programming*.

To fully understand the content of this notebook, the reader should:

* Be familiar with Python.
* Have a background in any branch of engineering, computer science, data science, economics, statistics, any branch of the “hard” sciences, or any discipline that uses quantitative models and methods.

The reader should also review the  [documentation](https://www.gurobi.com/resources/?category-filter=documentation)
of the Gurobi Python API.

**Download the Repository** <br />
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

## The Furniture Factory Problem

A data scientist is in charge of developing the weekly production plan for two key products that the furniture factory makes: chairs and tables.  Using machine learning techniques, the data scientist predicts that the selling price of a chair is $\$45$ and the selling price of a table is $\$80$ dollars. There are two critical resources in the production of chairs and tables:
mahogany (measured in board square-feet) and labor (measured in work hours). There are 400 units of mahogany available at the beginning of each week, and there are 450 units of labor available during each week.

![data_scientist](data_scientist.PNG)

The data scientist estimates that the production of one chair requires 5 units of mahogany and 10 units of labor and the production of one table requires 20 units of mahogany and 15 units of labor. The marketing department has told the data scientist that all chairs and tables that are produced will be sold.

![BOM](BOM.PNG)

Our goal is to create a production plan that maximizes total revenue.

To create such a production plan, we need to decide how many chairs and tables to make in order to maximize total revenue, while satisfying resource constraints. This problem has two decision variables:
* x1: number of chairs to produce.
* x2: number of tables to produce. 

The number of chairs and tables to produce should be a non-negative number. That is, x1, x2 ≥ 0. The data of this furniture problem is presented in the table below. 

![problem_summary](problem_summary.PNG)

If we know the amount of chairs to produce (x1), then since each chair generates $\$45$, the total revenue generated by the production of chairs can be determined by the term 45x1 ($45*x1$). 

Similarly, the total revenue generated by the production of tables can be determined by the term 80x2 ($80*x2$). Consequently, the total revenue generated by the production plan can be determined by the following equation.

$$
\text{Revenue} = 45x1 + 80x2
$$

The production plan is constrained by the amount of resources available. How do we ensure that the production plan does not require more mahogany than the amount of mahogany available? If we decide to produce x1 number of chairs, then the total amount of mahogany required for the production of chairs is 5x1 ($5*x1$). Similarly, if we decide to produce x2 number of tables, then the total amount of mahogany required for the production of tables is 20x2 ($20*x2$).

Hence, the total consumption of mahogany by the production plan, determined by the values of x1 and x2. is ($5x1 + 20x2$). However, the consumption of mahogany by the production plan cannot exceed the amount of mahogany available. We can express these ideas in the following constraint:

$$
5x1 + 20x2 \leq 400
$$

In similar fashion, we can formulate the constraint for labor resources.

The total amount of labor resources required for the production of chairs is 10 labor units multiplied by the number of chairs produced, that is 10x1 ($10*x1$). The total amount of labor resources required for the production of tables is 15 labor units multiplied by the number of tables produced, that is 15x2 ($15*x2$). Therefore, the total consumption of labor resources by the production plan, determined by the values of x1 and x2, is 
($10x1 + 15x2$). This labor consumption cannot exceed the labor capacity available. Hence, this constraint can be expressed as follows:

$$
10x1 + 15x2 \leq 450
$$

In summary, the linear programming formulation of the Furniture Factory Problem follows.

$$
\text{Max} \; \text{Revenue} = 45x_{1} + 80x_{2}
$$

Subject to the following constraints:
$$
\text{Mahogany} : 5x_{1} + 20x_{2} \leq 400
$$

$$
\text{Labor} : 10x_{1} + 15x_{2} \leq 450
$$

$$
\text{Non-negativity} : x_{1}, x_{2} \geq 0 
$$ 

In [1]:
# pip install for a limited license of the Gurobi callable library
# %pip install gurobipy

## Python Implementation

The following Python code imports the Gurobi callable library and imports the `GRB` class into the main namespace.

In [2]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

The `Model()` constructor creates a model object f . The name of this model is ‘Furniture’. This model f initially contains no decision variables, constraints, or objective function.

In [3]:
# Create Furniture Factory  model
f = gp.Model("Furniture")

Using license file C:\Users\panit\gurobi.lic


The `f.addVar()` method adds a decision variable to the model object f, one by one; i.e. x1 and  then x2. The argument of the method gives the name of added decision variable. The default values are applied here; i.e. the decision variables are of type continuous and non-negative, with no upper bound.

In [4]:
# Create decision variables
x1 = f.addVar(name="chairs")
x2 = f.addVar(name="tables")

The `f.setObjective()` method adds the objective function to the model object f. The first argument is a linear expression (LinExpr) and the second argument defines the sense of the optimization. A linear expression object (LinExpr) consists of a constant term, plus a sum of coefficient-variables pairs that capture the linear terms.

In [5]:
# Define objective function: Maximize total revenue
f.setObjective(45*x1 + 80*x2, GRB.MAXIMIZE)

The `f.addConstr()` method adds a constraint to the model object f and considers a linear expression (LinExpr) as the left-hand-side of the constraints, the sense of the constraint, and its capacity value. The last argument gives the name of the constraint. The `f.addConstr()` method returns a Gurobi tupledict object that contains the constraint recently created. The Gurobi tupledict object for the first constraint is called mahogany, and for the second constraint is called labor. Notice that we are using the same name for the Gurobi tupledict object and the name of the constraint.

In [6]:
# Add mahogany constraint
mahogany = f.addConstr(5*x1 + 20*x2 <= 400, name="mahogany")
# add labor constraint
labor = f.addConstr(10*x1 + 15*x2 <= 450, name="labor")

In [7]:
# save model for inspection
f.write('furniture.lp')

![furniture](furniture.PNG)

The `f.optimize()` method runs the optimization engine to solve the LP problem in the model object f

In [8]:
# Run optimization engine
f.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xd0437183
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  Objective range  [5e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 5e+02]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5000000e+31   2.968750e+30   6.500000e+01      0s
       2    2.2000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  2.200000000e+03


A brief description of the output report of the Gurobi Optimizer follows. 

The first line of the output report describes the version of Gurobi used to solve the optimization model.

The second line describes number of threads used by the Gurobi Optimizer.

The third line display the number of rows, columns, and non-zero coefficients of the optimization model.

The "coefficients statistics" describe important value ranges of the parameters of the model. 
* The "matrix range" defines the min/max absolute values of the matrix of coefficients of the LP model.
* The "objective range" determines the min/max absolute values of the objective function coefficients.
* The "bounds range" determines the min/max absolute values of the upper/lower bound values.
* The "RHS range" determines the min/max absolute values of the RHS (Right-Hand-Side) values.

These parameters value ranges are important because "wide" ranges may cause numerical instability during the solution of the linear programming (LP) problem.

When the `Presolve` algorithm is invoked, these output report displays the size of the reduced optimization model. Notice that for the Furniture Factory Problem, `Presolve` does not reduced the size of the model.

The next table describes:
* The number of iterations to solve the LP model.
* The value of the objective function at each iteration.
* The magnitude of the Primal LP model infeasibility.
* The magnitude of the Dual LP model infeasibility.
* The elapsed time at each iteration.

**Note**: Throughout this tutorial, you will learn what the Primal and Dual LP models are.

The last two lines define the number of iterations to solve the LP problem, the elapsed time to solve it, and the optimal value of the objective function.

In [9]:
# Display optimal production plan
for v in f.getVars():
    print(v.varName, v.x)

print(f"Optimal total revenue: ${f.objVal:,}")

chairs 24.0
tables 14.0
Optimal total revenue: $2,200.0


The `f.getVars()` method retrieves a list of all variables in the model object f. The print function displays the decision variable names `v.varName` and solution value `v.x`.

## Exercise

The data scientist wished to feed her children a breakfast menu which contains a specified level of nutrients. Using some predictive analytics techniques, she estimated that her children should obtain at least 1 milligram of thiamine, 5 milligrams of niacin, and 400 calories from their breakfast foods. 

The children have a choice of eating two brands of cereal: Krunchies and/or Crispies. The fine print on the side of each cereal box states the fact that 1 ounce of Krunchies contains  0.10 milligrams of thiamine, 1 milligram of niacin, and 110 calories; while 1 ounce of Crispies contains 0.25 milligrams of thiamine, 0.25 milligrams of niacin, and 120 calories. The cost per ounce of Krunchies is 38 cents and the cost per ounce of Crispies is 42 cents. The data scientist wants to find a breakfast diet for her children that satisfies the minimum nutrient requirements at a minimum cost.

The goal of this exercise is for you to formulate this diet problem as a linear programming problem using the Gurobi Python API and find the optimal diet.

## References

Saul I. Gass, An Illustrated Guide to Linear Programming, Dover Publications; Revised ed. edition (March 1, 1990).

Copyright © 2021 Gurobi Optimization, LLC